In Decision Tree the target variable is categorical (in Regression Tree the target variable is continuse)

In [13]:
# we use decision tree and random forost when the relationship between predictors and target is not linear (unlike logistic regression)
# check it with cross validation

In [14]:
from urllib.request import urlretrieve

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%203/data/drug200.csv'
output_file = 'drug200.csv'

import os
if not os.path.exists(output_file):
    urlretrieve(url, output_file)
else:
    pass

In [15]:
import pandas as pd
df = pd.read_csv(output_file)
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


In [16]:
# for decision tree we need to convert the categorical variables to numberical values with preprocessing.LabelEncoder()
# in knn, like linear regression, we use pd.get_dummies() for categoricals
# but in fact, in linear reg and knn it's better to use pd.get_dummies for nominal categoricals and use preprocessing.LabelEncoder() for ordinal categoricals
# unike tree based models that is always safe to use .LabelEncoder()
from sklearn.preprocessing import LabelEncoder

X_cont = df[['Age', 'Na_to_K']]
X_cat = df[['Sex', 'BP', 'Cholesterol']]
y = df['Drug'].values

# instead of LabelEncode each one separately (in the main file), we can do it in just one line
X_cat_labeled = X_cat.apply(LabelEncoder().fit_transform)
X = pd.concat([X_cont, X_cat_labeled], axis=1)

from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)  # for decision tree it is not needed but doesn't make any changes too, or decision tree it is not needed but doesn't make any changes too, but since we need it for cross validation we do it anywasy

In [21]:
# cross validatoin to see if we should use logistic regression or decision tree (or random forest)
# check the decisionTree_mine for cases where we have categorical variables
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

log_scores = cross_val_score(LogisticRegression(), X, y, cv=5)
tree_scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)

print("Logistic Regression average score:", log_scores.mean())
print("Decision Tree average score:", tree_scores.mean())

# If decision tree is better by > 0.05 on avergae score, go with the decision tree or random forest
# otherwise go with logistic regression

# If logistic regression performs similarly to or better than the decision tree, it suggests your relationships are mostly linear. 
# If the decision tree performs notably better, you likely have non-linear relationships.

Logistic Regression average score: 0.93
Decision Tree average score: 0.985


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_dTree = DecisionTreeClassifier(criterion='entropy', max_depth=4).fit(X_train, y_train)
# max_depth:
# Small dataset (roughly < 1000 rows): Start with max_depth = 3-5
# Medium dataset (roughly 1000-10,000 rows): Try max_depth = 6-10
# Large dataset (roughly > 10,000 rows): Can try max_depth > 10
# cahnge it untill get the max test accuracy?

y_pred = model_dTree.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

1.0

In [19]:
# Vizualize
from sklearn.tree import export_graphviz

export_graphviz(model_dTree, out_file='tree.dot', filled=True, feature_names=['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K'])